In [2]:
import pickle 
from importlib import reload

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import models.custom.detector as g
from libs.dataset_utils import prepare_data_from_tfrecord
from libs.image_utils import (draw_bounding_box,
                              calc_iou_accuracy,
                              calc_iou_top_1_accuracy)
from libs.various_utils import load_from_pickle
from configs.project_config import project_path

from helpers import (evaluate,
                     visualize,
                     visualize_cam,
                     restore_preprocessed)

In [9]:
"""
Constant
"""
NUM_CLASS = 200
INPUT_SHAPE = (64, 64, 3)
MODEL_BASE_INPUT_SHAPE = (224, 224, 3)

tfrecord_train_dir = "{}/data/tiny-imagenet-200/tfrecord/train".format(project_path)
tfrecord_valid_dir = "{}/data/tiny-imagenet-200/tfrecord/valid".format(project_path)
tfrecord_test_dir = "{}/data/tiny-imagenet-200/tfrecord/test".format(project_path)

meta_path = "{}/data/tiny-imagenet-200/meta.pickle".format(project_path)
pretrained_ckpt_path = "{}/checkpoints/inception_v3/inception_v3.ckpt".format(project_path)

vanila_model_save_path = "{}/checkpoints/vanila_inception_v3/vanila_inception_v3".format(project_path)

pickle_data_full_path = "{}/data/tiny-imagenet-200/pickle/tiny_imagenet.pickle".format(project_path)

# Build Model, Dataset Pipeline and Load Pretrained Inception Weights

In [5]:
# ==============================================================================
# model_base_name : { "InceptionV3", "InceptionV2", "alexnet_v2"}
# 각 모델마다 final endpoint를 변경할 수 있다. final endpoint 의 정보는 models/안에 모델별 폴더안의 python 파일에 들어있다.
model_base_name = "InceptionV3"
model = g.Detector(output_dim=NUM_CLASS,
                   input_shape=INPUT_SHAPE,
                   model_base_input_shape=MODEL_BASE_INPUT_SHAPE,
                   model_base_name=model_base_name,
                   model_base_final_endpoint='Mixed_7c',
                   model_name="weakly_detector")

X_preprocessed shape: [None, 224, 224, 3]
last_conv shape : [None, 5, 5, 1024]
gap shape :[None, 1, 1, 1024]
h shape :[None, 1, 1, 1024]
logits_before shape :[None, 1, 1, 200]
logits shape :[None, 200]
W shape :[1, 1, 1024, 200]
class_weight_by_row shape : [200, 1024]
batch_class_weight shape : [None, 1024, 1]
last_conv_flatten_w_h shape : [None, 4096, 1024]
clf_var_to_optimize_list
[<tf.Variable 'clf/conv/weights:0' shape=(3, 3, 2048, 1024) dtype=float32_ref>,
 <tf.Variable 'clf/conv/BatchNorm/beta:0' shape=(1024,) dtype=float32_ref>,
 <tf.Variable 'clf/conv/BatchNorm/moving_mean:0' shape=(1024,) dtype=float32_ref>,
 <tf.Variable 'clf/conv/BatchNorm/moving_variance:0' shape=(1024,) dtype=float32_ref>,
 <tf.Variable 'clf/logits/weights:0' shape=(1, 1, 1024, 200) dtype=float32_ref>,
 <tf.Variable 'clf/logits/biases:0' shape=(200,) dtype=float32_ref>]


In [6]:
"""
Read Data Pickle
"""
try :
    d = load_from_pickle(pickle_data_full_path)
    (X_train, P_train, Y_train, Y_train_one_hot) = (d['X_train'], d['P_train'], d['Y_train'] ,d['Y_train_one_hot'])
    (X_valid, P_valid, Y_valid, Y_valid_one_hot) = (d['X_valid'], d['P_valid'], d['Y_valid'] ,d['Y_valid_one_hot'])
    (X_test, P_test, Y_test, Y_test_one_hot) = (d['X_test'], d['P_test'], d['Y_test'] ,d['Y_test_one_hot'])
    (idx_word_dict, word_idx_dict) = (d['idx_word_dict'], d['word_idx_dict'])
    (idx_nid_dict, nid_idx_dict) = (d['idx_nid_dict'], d['nid_idx_dict'])
except Exception as e: 
    print("maybe there is no pickle data: {}".format(e))

In [10]:
# ==============================================================================
with model.g.as_default():
    """
    Read Data
    """
    d = prepare_data_from_tfrecord(
        tfrecord_train_dir=tfrecord_train_dir,
        tfrecord_valid_dir=tfrecord_valid_dir,
        tfrecord_test_dir=tfrecord_test_dir,
        batch_size=64,
        shuffle_buffer_size=1000)
    (X, Y, P,
     init_dataset_train,
     init_dataset_valid,
     init_dataset_test,
     
     X_raw, Y_raw, P_raw,
     init_dataset_train_raw,
     init_dataset_valid_raw,
     init_dataset_test_raw,) = (d['X'], d['Y'], d['P'],
                            d['init_dataset_train'],
                            d['init_dataset_valid'],
                            d['init_dataset_test'],
                            d['X_raw'], d['Y_raw'], d['P_raw'],
                            d['init_dataset_train_raw'],
                            d['init_dataset_valid_raw'],
                            d['init_dataset_test_raw'])

    meta = load_from_pickle(meta_path)
    model.meta.update(meta)

    """
    Initialize with pretrained weights
    """
    variables_to_restore = tf.contrib.framework.get_variables_to_restore(
        include=[model_base_name])
    init_pretrain_fn = tf.contrib.framework.assign_from_checkpoint_fn(
        pretrained_ckpt_path, variables_to_restore)

    init_pretrain_fn(model.sess)

INFO:tensorflow:Restoring parameters from /home/sungju/projects/sj/weakly_supervised_detector/checkpoints/inception_v3/inception_v3.ckpt


# Vanila Model - Learning Deep Features for Discriminative Localization

In [11]:
# ==============================================================================

model.train_with_dataset_api(X=X,
                             Y=Y,
                             init_dataset_train=init_dataset_train,
                             init_dataset_valid=init_dataset_valid,
                             n_epoch=2,
                             learning_rate=0.001,
                             reg_lambda=0.,
                             dropout_keep_prob=0.8,
                             patience=10,
                             verbose_interval=1,
                             mode=g.MODE_TRAIN_ONLY_CLF,
                             save_dir_path=None)

epoch 0, batch 1249 takes: 0.30 sec:   0%|          | 0/2 [08:42<?, ?it/s]

------------------------------
epoch_i : 0
train loss: 14.333846949386597, train accuracy: 0.571075
valid loss: 15.448203465270996, valid accuracy: 0.55925
best valid loss: 999999999.0, best valid accuracy : 0.0


epoch 0, batch 1249 takes: 0.30 sec:  50%|█████     | 1/2 [17:46<17:46, 1066.39s/it]

******************************
epoh_i : 0
train loss: 14.333846949386597, train accuracy: 0.571075
valid loss: 15.448203465270996, valid accuracy: 0.55925
best valid loss: 15.448203465270996, best valid accuracy : 0.55925
save current model : ./tmp/20180906_005305_GKDWZB/weakly_detector


epoch 1, batch 1249 takes: 0.29 sec:  50%|█████     | 1/2 [26:24<26:24, 1584.03s/it]

------------------------------
epoch_i : 1
train loss: 17.705852726364135, train accuracy: 0.592325
valid loss: 20.06762677307129, valid accuracy: 0.57235
best valid loss: 15.448203465270996, best valid accuracy : 0.55925


epoch 1, batch 1249 takes: 0.29 sec: 100%|██████████| 2/2 [35:23<00:00, 1061.93s/it]

******************************
epoh_i : 1
train loss: 17.705852726364135, train accuracy: 0.592325
valid loss: 20.06762677307129, valid accuracy: 0.57235
best valid loss: 20.06762677307129, best valid accuracy : 0.57235
save current model : ./tmp/20180906_005305_GKDWZB/weakly_detector
train takes : 2123.8634312152863 sec
INFO:tensorflow:Restoring parameters from ./tmp/20180906_005305_GKDWZB/weakly_detector


******************************
final trained performance
train loss: 17.705852561569213, train accuracy: 0.592325
valid loss: 20.06762677307129, valid accuracy: 0.57235
best valid loss: 20.06762677307129, best valid accuracy : 0.57235
final_model_path: ./tmp/20180906_005305_GKDWZB/20180906_013727_final_weakly_detector
train done
******************************


In [12]:
model.train_with_dataset_api(X=X,
                             Y=Y,
                             init_dataset_train=init_dataset_train,
                             init_dataset_valid=init_dataset_valid,
                             n_epoch=1,
                             learning_rate=0.001,
                             reg_lambda=0.,
                             dropout_keep_prob=0.8,
                             patience=10,
                             verbose_interval=1,
                             mode=g.MODE_TRAIN_GLOBAL,
                             save_dir_path=None)

model.save(vanila_model_save_path)

  0%|          | 0/1 [00:00<?, ?it/s]

******************************
Make directory with save_dir_path is failed
Maybe, there is directory already or error because of "[Errno 17] File exists: './tmp/20180906_005305_GKDWZB'"


ResourceExhaustedError: OOM when allocating tensor with shape[64,12,12,768]
	 [[Node: InceptionV3/Mixed_6d/Branch_3/AvgPool_0a_3x3/AvgPool = AvgPool[T=DT_FLOAT, data_format="NHWC", ksize=[1, 3, 3, 1], padding="SAME", strides=[1, 1, 1, 1], _device="/job:localhost/replica:0/task:0/device:GPU:0"](InceptionV3/Mixed_6c/concat)]]

Caused by op 'InceptionV3/Mixed_6d/Branch_3/AvgPool_0a_3x3/AvgPool', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.5/dist-packages/ipykernel/kernelapp.py", line 478, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.5/dist-packa

# Run Sample

In [ ]:
# ==============================================================================
num_sample = 100
X_sample =  X_train[:num_sample]
P_sample = P_train[:num_sample]
Y_sample_one_hot = Y_train_one_hot[:num_sample]
Y_sample = Y_train[:num_sample]

In [ ]:
(cam_list_sample,
bbox_list_sample,
gt_known_loc_accuracy_sample,
top_1_loc_accuracy_sample) = evaluate(model, X_sample, P_sample, Y_sample_one_hot, 'sample')

In [ ]:
visualize_cam(X_sample, Y_sample, cam_list_sample, idx_word_dict, n_show=2, start=16)

# Evaluation

In [ ]:
(cam_list_train,
bbox_list_train,
gt_known_loc_accuracy_train,
top_1_loc_accuracy_train) = evaluate(model, X_train, P_train, Y_train_one_hot, 'train',
                                     flag_preprocess=False)

In [ ]:
(cam_list_valid,
bbox_list_valid,
gt_known_loc_accuracy_valid,
top_1_loc_accuracy_valid) = evaluate(model, X_valid, P_valid, Y_valid_one_hot, 'valid',
                                     flag_preprocess=False)

In [ ]:
(cam_list_test,
bbox_list_test,
gt_known_loc_accuracy_test,
top_1_loc_accuracy_test) = evaluate(model, X_test, P_test, Y_test_one_hot, 'test',
                                    flag_preprocess=False)

# Visualization

In [ ]:
visualize(X_train, P_train, Y_train, cam_list_train, bbox_list_train, idx_word_dict,
          n_show=3, start=0)

In [ ]:
visualize(X_valid, P_valid, Y_valid, cam_list_valid, bbox_list_valid, idx_word_dict,
          n_show=3, start=0)

In [ ]:
visualize(X_test, P_test, Y_test, cam_list_test, bbox_list_test, idx_word_dict,
          n_show=3, start=6)